In [36]:
import pandas as pd
from selenium import webdriver 
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  
from selenium.common.exceptions import NoSuchElementException  #Este importé para manejar más errores.
from time import sleep

In [37]:
df_peliculas = pd.read_csv("peliculas.csv")

df_peliculas.head()


,Tipo,Nombre,Año de Estreno,Mes de Estreno,ID,Genero
0,Movie,Pál Adrienn,2011,6,tt0146592,Accion
1,Movie,Oda az igazság,2010,2,tt0154039,Accion
2,Movie,A zöld sárkány gyermekei,2011,4,tt0162942,Accion
3,Movie,The Final Journey,2010,9,tt0230212,Accion
4,Movie,In My Sleep,2010,4,tt0326965,Accion


In [38]:

df_peliculas.tail()

,Tipo,Nombre,Año de Estreno,Mes de Estreno,ID,Genero
22495,Movie,Reborn,2018,8,tt4902266,Comedia
22496,Movie,The Further Adventures of Walt's Frozen Head,2019,3,tt4906764,Comedia
22497,Movie,I'm Every Woman,2018,11,tt4908280,Comedia
22498,Movie,Bernadette,2018,4,tt4921954,Comedia
22499,Movie,CHIC: Part Deux,2018,7,tt4925796,Comedia


In [39]:
# Guardar el DataFrame combinado en un nuevo archivo CSV
df_peliculas.to_csv('peliculas.csv', index=False)

In [40]:
df_peliculas

,Tipo,Nombre,Año de Estreno,Mes de Estreno,ID,Genero
0,Movie,Pál Adrienn,2011,6,tt0146592,Accion
1,Movie,Oda az igazság,2010,2,tt0154039,Accion
2,Movie,A zöld sárkány gyermekei,2011,4,tt0162942,Accion
3,Movie,The Final Journey,2010,9,tt0230212,Accion
4,Movie,In My Sleep,2010,4,tt0326965,Accion
...,...,...,...,...,...,...
22495,Movie,Reborn,2018,8,tt4902266,Comedia
22496,Movie,The Further Adventures of Walt's Frozen Head,2019,3,tt4906764,Comedia
22497,Movie,I'm Every Woman,2018,11,tt4908280,Comedia
22498,Movie,Bernadette,2018,4,tt4921954,Comedia


In [41]:
df_peliculas.columns

Index(['Tipo', 'Nombre', 'Año de Estreno', 'Mes de Estreno', 'ID', 'Genero'], dtype='object')

In [42]:
df_peliculas['ID']

0        tt0146592
1        tt0154039
2        tt0162942
3        tt0230212
4        tt0326965
           ...    
22495    tt4902266
22496    tt4906764
22497    tt4908280
22498    tt4921954
22499    tt4925796
Name: ID, Length: 22500, dtype: object

In [43]:
id_peliculas = df_peliculas['ID']
id_peliculas.head(20)

0     tt0146592
1     tt0154039
2     tt0162942
3     tt0230212
4     tt0326965
5     tt0331312
6     tt0337882
7     tt0393049
8     tt0396123
9     tt0402887
10    tt0403645
11    tt0411951
12    tt0419692
13    tt0443331
14    tt0445054
15    tt0444531
16    tt0446686
17    tt0451108
18    tt0454155
19    tt0457591
Name: ID, dtype: object

## Extraccion datos imdb:

In [44]:
id_peliculas
driver = webdriver.Chrome()
driver.get("https://www.imdb.com")
sleep(1)
driver.maximize_window()
sleep(1)
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
sleep(1)
diccionario_imdb = {
        "puntuacion": [],
        "direccion": [],
        "guion": [],
        "sinopsis": [],
        "duracion": [],
        "nombre": []}

for id in id_peliculas[ 1:5]:
    try:
        barra_busqueda = driver.find_element("css selector", "#suggestion-search")
        # barra_busqueda.clear()  # Limpiar cualquier texto anterior
        sleep(1)
        barra_busqueda.send_keys(id,Keys.ENTER)
        sleep(2)
        
        
        if id not in driver.current_url: # Esto es una herramienta de Selenium para indicar en que pagina estamos actualmente.
            print(f"ID {id} de película no encontrado, seguimos con el siguiente.")
            continue
        
        diccionario_imdb["puntuacion"].append(driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-4e4cc5f9-3.dDRspk > div.sc-3a4309f8-0.bjXIAP.sc-b7c53eda-1.iIQkEw > div > div:nth-child(1) > a > span > div > div.sc-bde20123-0.dLwiNw > div.sc-bde20123-2.cdQqzc > span.sc-bde20123-1.cMEQkK").text)
        diccionario_imdb["direccion"].append(driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-4e4cc5f9-4.ixQHwZ > div.sc-4e4cc5f9-6.fFWQTt > div.sc-4e4cc5f9-10.jXLkbv > section > div.sc-b7c53eda-3.vXcqY > div > ul > li:nth-child(1) > div > ul > li").text)
        diccionario_imdb["guion"].append(driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-4e4cc5f9-4.ixQHwZ > div.sc-4e4cc5f9-6.fFWQTt > div.sc-4e4cc5f9-10.jXLkbv > section > div.sc-b7c53eda-3.vXcqY > div > ul > li:nth-child(2) > div").text)
        diccionario_imdb["sinopsis"].append(driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-4e4cc5f9-4.ixQHwZ > div.sc-4e4cc5f9-6.fFWQTt > div.sc-4e4cc5f9-10.jXLkbv > section > p > span.sc-a31b0662-2.geLkLc").text)
        diccionario_imdb["duracion"].append(driver.find_element("css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-4e4cc5f9-3.dDRspk > div.sc-b7c53eda-0.dUpRPQ > ul > li:nth-child(3)").text)
        diccionario_imdb["nombre"].append(driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-4e4cc5f9-3.dDRspk > div.sc-b7c53eda-0.dUpRPQ > h1 > span").text)
        print(f"Informacion extraida para el ID: {id}")
        
    except NoSuchElementException: #Herramienta importada de Selenium.
        print(f"No se encontro la pagina para el ID {id}")
    except Exception as e:
        print(f"Error durante la búsqueda del ID {id}: {str(e)}")
    finally:
        # Volver a la página de inicio para asegurar una búsqueda limpia
        driver.get("https://www.imdb.com")
        sleep(1)

driver.close()  


No se encontro la pagina para el ID tt0154039
No se encontro la pagina para el ID tt0162942
No se encontro la pagina para el ID tt0230212
Informacion extraida para el ID: tt0326965


In [45]:
diccionario_imdb

{'puntuacion': ['4,6', '6,9', '8,9', '5,5'],
 'direccion': ['Allen Wolf'],
 'guion': ['Allen Wolf'],
 'sinopsis': ['A sleepwalking sexaholic masseur wakes up covered in blood next to a bloody knife with cops ringing his doorbell.'],
 'duracion': ['1h 44min'],
 'nombre': ['Mientras duermo']}